# XGBoost

In [5]:
# https://www.kaggle.com/c/titanic

In [6]:
import sklearn
import pandas as pd

In [7]:
# Загружаем данные из файлов
train = pd.read_csv('./data/titanic/train.csv')
test = pd.read_csv('./data/titanic/test.csv')

In [8]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Предобработка данных

In [9]:
# Заполняем пропуски в данных медианными 
# значениями факторов на обучающей выборке
train_median = train.median()
train_imp = train.fillna(train_median)
test_imp = test.fillna(train_median)

In [10]:
# Бинаризуем категориальные признаки
CATEGORY_COL = ['Sex', 'Pclass', 'Embarked']
train_dummies = pd.get_dummies(train_imp, columns=CATEGORY_COL, drop_first=True)
test_dummies = pd.get_dummies(test_imp, columns=CATEGORY_COL, drop_first=True)

In [11]:
train_dummies.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,0,0,0,1
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,1,0,1


In [12]:
# Удаляем лишние столбцы
DROP_COL = ['PassengerId', 'Name', 'Ticket', 'Cabin']
TARGET_COL = 'Survived'
X_train = train_dummies.drop(DROP_COL + [TARGET_COL], axis=1)
y_train = train_dummies[TARGET_COL]
X_test = test_dummies.drop(DROP_COL, axis=1)

# XGBoost

**Основные параметры**

| Параметр          | Тип                | Описание                                                                                                                                     | Диапазон значений      |
|-------------------|--------------------|----------------------------------------------------------------------------------------------------------------------------------------------|------------------------|
| **max_depth**         | int                | Максимальная глубина каждого дерева                                                                                                          | 3-10                   |
| **learning_rate**     | float              | Шаг градиентного бустинга                                                                                                                    | 0.01-0.2               |
| **n_estimators**      | int                | Размер ансамбля                                                                                                                              | 100-1000               |
| **objective**         | string or callable | Оптмизируемый функционал. В отличие от метрики качества, должен быть всюду дифференцируем                                                    |                        |
| **booster**           | string             | Какой бустинг использовать                                                                                                                   | [gbtree,gblinear,dart] |
| **nthread**           | int                | Количество потоков, в котором будет обучаться XGBoost                                                                                        |                        |
| **n_jobs**            | int                | -//-                                                                                                                                         |                        |
| **gamma**             | float              |  Минимальный прирост качества функционала ошибки, при котором лист дерева будет поделён на поддеревья. Минимальный критерий ветвления дерева |                        |
| **min_child_weight**  | float              | Минимальная сумма весов в листе дерева для того, чтобы его не выкинуть                                                                       |                        |
| **max_delta_step**    | int                | Максимальный шаг, с которым можно обновлять веса деревьев                                                                                    |                        |
| **subsample**         | float              | На какой случайной доли обучающей выборки будет обучаться каждое дерево                                                                      |                        |
| **colsample_bytree**  | float              | На какой случайной доли признаков будет обучаться каждое дерево                                                                              | [0.5-1]                |
| **colsample_bylevel** | float              | Какая доля признаков пойдёт в каждое поддерево                                                                                               |                        |
| **reg_alpha**         | float              | L1-регуляризация для листьев дерева                                                                                                          |                        |
| **reg_lambda**        | float              | L2-регуляризация для листьев дерева                                                                                                          |                        |
| **scale_pos_weight**  | float              | Какой вес придадим второму классу объектов (1) по сравнению с первым (0). Используется при сильно несбалансированной выборке                 |                        |
| **base_score**        | float              |  До какого скора объекты будут отнесену к первому классу (0) и с какого ко второму (1). По умолчанию 0.5. Обычно не тюнят                    |                        |
| **seed**              | int                | Начальная точка для генератора случайных чисел                                                                                               |                        |
| **random_state**      | int                | -//-                                                                                                                                         |                        |
| **missing**           | float (optional)   | Что принимать за пропущенные значения. По умолчанию np.nan.                                                                                  |                        |
| **silent**            | boolean            | Если True, то выводить всю информацию об обучении                                                                                            |                        |

# Упражнение 1
Используя текущее описание параметров и предыдущее упражнение, произведите базовый подбор параметров XGBoost'a, используя метрику **AUC**. Сравните полученное качество (**AUC**) с полученным ранее на RandomForest и GradientBoosting

In [ ]:
import xgboost

clf = xgboost.XGBClassifier()

## TODO: ваш GridSearch
...

# Более умный и последовательный тюнинг
Взято [отсюда](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/). Это наиболее полный и часто используемый мануал для тюнинга XGBoost.
**Алгоритм:**
1. Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate. XGBoost has a very useful function called as “cv” which performs cross-validation at each boosting iteration and thus returns the optimum number of trees required.
2. Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.
3. Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.
4. Lower the learning rate and decide the optimal parameters .

## Шаг 1: Зафиксируем learning_rate и параметры дерева и подберём n_estimators

Параметры:

* **max_depth**. Как указанов в таблице выше, обычно варьируется в интервале от 3 до 10 (но от задачи к задаче значения могут меняться). В качестве начального значения обычно используют 5
* **min_child_weight**. Если выборка сильно несбалансирована, то лучше выбрать значение "1". Иначе лучше выбрать значение "2" и зафиксировать
* **gamma**. Обычно выставляют значение в интервале от 0 до 0.2 и фиксируют. В дальнейшем этот параметр всегда можно затюнить отдельно
* **subsample, colsample_bytree**. Выставим 0.8 и зафиксируем. Можно также проварьировать в интервале 0.5-0.9.
* **scale_pos_weight**. Выставляется в зафисимости от соотношения классов в выборке и фиксируется

In [35]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

ones_ratio = y_train[y_train == 1].shape[0] * 1.0 / y_train[y_train == 0].shape[0] # посчитаем соотношение между классами

param_grid = {
    # параметры ансамбля
    'n_estimators': [10, 30, 50, 100, 200, 400, 600, 1000],
    'learning_rate': [0.1],
    
    # параметры дерева
    'max_depth': [5],
    'min_child_weight': [2],
    'gamma': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [ones_ratio],
    
    # параметры регуляризации
    'reg_alpha': [0.0],
    'reg_lambda': [1.0]
}

cv = KFold(n_splits=4, shuffle=True)

clf = xgboost.XGBClassifier()
gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=10, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=10, subsample=0.8, max_depth=5, gamma=0.1, score=0.843174, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=10, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=10, subsample=0.8, max_depth=5, gamma=0.1, score=0.813701, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=10, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_e

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
c:\users\artem\

[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=30, subsample=0.8, max_depth=5, gamma=0.1, score=0.810526, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=30, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=30, subsample=0.8, max_depth=5, gamma=0.1, score=0.919406, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=30, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=30, subsample=0.8, max_depth=5, gamma=0.1, score=0.864353, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, m

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version 

[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, max_depth=5, gamma=0.1, score=0.822348, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, max_depth=5, gamma=0.1, score=0.921591, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, max_depth=5, gamma=0.1 
[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=50, subsample=0.8, max_depth=5, gamma=0.1, score=0.861532, total=   0.0s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=100, subsample=0.8, 

...

[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=1000, subsample=0.8, max_depth=5, gamma=0.1, score=0.906206, total=   0.6s
[CV] colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=1000, subsample=0.8, max_depth=5, gamma=0.1 


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


[CV]  colsample_bytree=0.8, scale_pos_weight=0.622950819672, learning_rate=0.1, min_child_weight=2, n_estimators=1000, subsample=0.8, max_depth=5, gamma=0.1, score=0.873712, total=   0.7s
('Best score (AUC): ', 0.86652737057695028)
Best params: 


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    8.3s finished
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 2,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 2. Подбираем параметры дерева
* **max_depth** - будем варьировать от 3 до 10 с шагом 2
* **min_child_weight** - от 1 до 6 с шагом 2

In [36]:
param_grid = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}

clf = xgboost.XGBClassifier(**best_params) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] max_depth=3, min_child_weight=1 .................................


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklear

[CV] .. max_depth=3, min_child_weight=1, score=0.838931, total=   0.1s
[CV] max_depth=3, min_child_weight=1 .................................
[CV] .. max_depth=3, min_child_weight=1, score=0.889280, total=   0.0s
[CV] max_depth=3, min_child_weight=1 .................................
[CV] .. max_depth=3, min_child_weight=1, score=0.853873, total=   0.0s
[CV] max_depth=3, min_child_weight=1 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


[CV] .. max_depth=3, min_child_weight=1, score=0.873006, total=   0.0s


...

[CV] .. max_depth=9, min_child_weight=5, score=0.884020, total=   0.0s
[CV] max_depth=9, min_child_weight=5 .................................
[CV] .. max_depth=9, min_child_weight=5, score=0.860071, total=   0.0s
[CV] max_depth=9, min_child_weight=5 .................................
[CV] .. max_depth=9, min_child_weight=5, score=0.859248, total=   0.0s


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


('Best score (AUC): ', 0.86376221302649392)
Best params: 


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    5.5s finished
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 3. Подбираем gamma (критерий создания поддерева)
* **gamma** - от 0 до 0.5 с шагом 0.1

In [37]:
param_grid = {
    'gamma': [0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


Fitting 4 folds for each of 6 candidates, totalling 24 fits
[CV] gamma=0.0 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


[CV] ........................ gamma=0.0, score=0.884837, total=   0.1s
[CV] gamma=0.0 .......................................................
[CV] ........................ gamma=0.0, score=0.837937, total=   0.0s
[CV] gamma=0.0 .......................................................
[CV] ........................ gamma=0.0, score=0.867424, total=   0.0s
[CV] gamma=0.0 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


...

[CV] ........................ gamma=0.5, score=0.884962, total=   0.0s
[CV] gamma=0.5 .......................................................
[CV] ........................ gamma=0.5, score=0.842540, total=   0.0s
[CV] gamma=0.5 .......................................................
[CV] ........................ gamma=0.5, score=0.870338, total=   0.0s
[CV] gamma=0.5 .......................................................
[CV] ........................ gamma=0.5, score=0.877388, total=   0.0s


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


('Best score (AUC): ', 0.86894669147776737)
Best params: 


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    2.5s finished
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 4. Затюним subsample и colsample_bytree
* **subsample** - от 0.5 до 1.0 с шагом 0.1
* **colsample_bytree** - от 0.5 до 1.0 с шагом 0.1

In [38]:
param_grid = {
    'subsample': [0.5 + 0.1*i for i in range(6)],
    'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklear

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] subsample=0.5, colsample_bytree=0.5 .............................
[CV]  subsample=0.5, colsample_bytree=0.5, score=0.871287, total=   0.0s
[CV] subsample=0.5, colsample_bytree=0.5 .............................
[CV]  subsample=0.5, colsample_bytree=0.5, score=0.809761, total=   0.0s
[CV] subsample=0.5, colsample_bytree=0.5 .............................
[CV]  subsample=0.5, colsample_bytree=0.5, score=0.893521, total=   0.0s
[CV] subsample=0.5, colsample_bytree=0.5 .............................
[CV]  subsample=0.5, colsample_bytree=0.5, score=0.906079, total=   0.0s
[CV] subsample=0.6, colsample_bytree=0.5 .............................
[CV]  subsample=0.6, colsample_bytree=0.5, score=0.871754, total=   0.0s
[CV] subsample=0.6, colsample_bytree=0.5 .............................
[CV]  subsample=0.6, colsample_bytree=0.5, score=0.821483, total=   0.0s
[CV] subsample=0.6, colsample_bytree=0.5 .............................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\

[CV]  subsample=0.6, colsample_bytree=0.5, score=0.895610, total=   0.0s
[CV] subsample=0.6, colsample_bytree=0.5 .............................
[CV]  subsample=0.6, colsample_bytree=0.5, score=0.905267, total=   0.0s
[CV] subsample=0.7, colsample_bytree=0.5 .............................
[CV]  subsample=0.7, colsample_bytree=0.5, score=0.869250, total=   0.0s
[CV] subsample=0.7, colsample_bytree=0.5 .............................
[CV]  subsample=0.7, colsample_bytree=0.5, score=0.812788, total=   0.0s
[CV] subsample=0.7, colsample_bytree=0.5 .............................


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version 

...

[CV]  subsample=0.9, colsample_bytree=1.0, score=0.898683, total=   0.0s
[CV] subsample=1.0, colsample_bytree=1.0 .............................
[CV]  subsample=1.0, colsample_bytree=1.0, score=0.869207, total=   0.0s
[CV] subsample=1.0, colsample_bytree=1.0 .............................
[CV]  subsample=1.0, colsample_bytree=1.0, score=0.811978, total=   0.0s
[CV] subsample=1.0, colsample_bytree=1.0 .............................


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


[CV]  subsample=1.0, colsample_bytree=1.0, score=0.895652, total=   0.0s
[CV] subsample=1.0, colsample_bytree=1.0 .............................
[CV]  subsample=1.0, colsample_bytree=1.0, score=0.898512, total=   0.0s
('Best score (AUC): ', 0.87715398071888762)
Best params: 


[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:   12.5s finished
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.6}

## Шаг 5. Регуляризация
* **reg_alpha** [1e-5, 1e-2, 0.1, 1, 100]
* **reg_lambda** [1e-5, 1e-2, 0.1, 1, 100]

In [40]:
param_grid = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklear

Fitting 4 folds for each of 25 candidates, totalling 100 fits
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.857702, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.854033, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.871397, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.878441, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.853241, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.853249, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.874271, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.878615, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.1 .................................


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


...

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version 

[CV] .... reg_alpha=100, reg_lambda=100, score=0.500000, total=   0.0s
[CV] reg_alpha=100, reg_lambda=100 ...................................
[CV] .... reg_alpha=100, reg_lambda=100, score=0.500000, total=   0.0s


c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    8.8s finished
c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


('Best score (AUC): ', 0.86688430999798638)
Best params: 


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 1,
 'reg_lambda': 0.01,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.6}

## Шаг 6. Learning rate
Чем меньше у нас **n_estimators** в ансамбле, тем быстрее нам нужно двигаться с каждым шагом (добавлением нового классификатора), т.е. делать больший **learning_rate**. Обычно **learning rate** варьируют так, чтобы произведение **n_estimators** x **learning_rate** оставалось инвариантным

In [43]:
import numpy as np
from sklearn.metrics import roc_auc_score

clf = xgboost.XGBClassifier(**best_params)
best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
best_learning_rate = best_params['learning_rate'] # аналогичная запись
invariant_composition = best_n_estimators * best_learning_rate
n_estimators_range = [10, 30, 100, 200, 400, 600, 800, 1000]

best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

for n_estimators in n_estimators_range:
    learning_rate = invariant_composition / n_estimators
    clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
    aucs = []
    for train_idx, test_idx in cv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
    auc = np.mean(auc)
    if auc > best_score:
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_score = auc
        
best_params['n_estimators'] = best_n_estimators
best_params['learning_rate'] = best_learning_rate

print('Best score (AUC): ', best_score)
print('Best params: ')
best_params

c:\users\artem\anaconda3\envs\lasagne2.7\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


('Best score (AUC): ', 0.92546693812516601)
Best params: 


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.4,
 'learning_rate': 0.0125,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 800,
 'n_jobs': 1,
 'nthread': 1,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 1,
 'reg_lambda': 0.01,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.6}

# Выведем важность признаков

In [47]:
for feature_name, feature_importance in zip(X_train.columns, clf.feature_importances_):
    print('Feature: "%s"\tFeature importance: %.4f' % (feature_name, feature_importance))

Feature: "Age"	Feature importance: 0.2828
Feature: "SibSp"	Feature importance: 0.0719
Feature: "Parch"	Feature importance: 0.0415
Feature: "Fare"	Feature importance: 0.3659
Feature: "Sex_male"	Feature importance: 0.0969
Feature: "Pclass_2"	Feature importance: 0.0210
Feature: "Pclass_3"	Feature importance: 0.0667
Feature: "Embarked_Q"	Feature importance: 0.0102
Feature: "Embarked_S"	Feature importance: 0.0431


# HyperOpt

In [48]:
!pip install hyperopt
!pip install networkx==1.11

^C


In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# в этой функции мы проверяем, как ведёт себя модель при заданных параметрах
def score(params):
    print("Training with params : ")
    print(params)
    params['n_estimators'] = int(params['n_estimators'])
    model = xgboost.XGBClassifier(**params)
    aucs = []
    # Для оценки качества используем KFold, который определили выше
    for train_idx, test_idx in cv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
    auc = np.mean(auc)
    result = {'loss': auc, 'status': STATUS_OK}
    print(result)
    return result


# это наша главная функция, в которой мы задаём параметры
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 5), # (название параметра, от, до, шаг)
             'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 10, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'objective': 'reg:linear',
             'silent' : 1,
             'scale_pos_weight': hp.quniform('scale_pos_weight', 0.5, 10., 0.5),
             'reg_alpha': 0.0,
             'reg_lambda': 1.0
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)
    print('Best: ')
    print(best)

#сюда будет записана
trials = Trials()

optimize(trials)

Training with params : 
{'reg_alpha': 0.0, 'colsample_bytree': 0.8500000000000001, 'silent': 1, 'scale_pos_weight': 9.5, 'learning_rate': 0.2, 'min_child_weight': 3.0, 'n_estimators': 270.0, 'subsample': 0.9500000000000001, 'reg_lambda': 1.0, 'objective': 'reg:linear', 'max_depth': 7.0, 'gamma': 0.75}
{'status': 'ok', 'loss': 0.9290969899665551}
Training with params : 
{'reg_alpha': 0.0, 'colsample_bytree': 0.6000000000000001, 'silent': 1, 'scale_pos_weight': 8.0, 'learning_rate': 0.5, 'min_child_weight': 4.0, 'n_estimators': 770.0, 'subsample': 0.9500000000000001, 'reg_lambda': 1.0, 'objective': 'reg:linear', 'max_depth': 7.0, 'gamma': 0.7000000000000001}
{'status': 'ok', 'loss': 0.83310433662516103}
Training with params : 
{'reg_alpha': 0.0, 'colsample_bytree': 0.7000000000000001, 'silent': 1, 'scale_pos_weight': 2.0, 'learning_rate': 0.375, 'min_child_weight': 3.0, 'n_estimators': 645.0, 'subsample': 0.8, 'reg_lambda': 1.0, 'objective': 'reg:linear', 'max_depth': 5.0, 'gamma': 0.600